In [21]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import joblib

In [22]:
# Paths to directories for datasets
train_combined_dir = r"C:\Users\grand\Downloads\Dataset\Train\Combined"
validation_combined_dir = r"C:\Users\grand\Downloads\Dataset\Validation\Combined"
test_combined_dir = r"C:\Users\grand\Downloads\Dataset\Test\Combined"

In [23]:
# Image Data Generators (Rescale pixel values and augment)
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Load Pre-trained VGG16 Model (used for feature extraction)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
model = Model(inputs=base_model.input, outputs=base_model.output)

In [24]:
# Function to Create Image Generators
def create_train_generator():
    """Create image generator for training."""
    return train_datagen.flow_from_directory(
        train_combined_dir,  
        target_size=(224, 224),  
        batch_size=32,
        class_mode='categorical',
        shuffle=True,
        seed=42  # Set seed for reproducibility
    )

def create_validation_generator():
    """Create image generator for validation."""
    return valid_datagen.flow_from_directory(
        validation_combined_dir,
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical',
        shuffle=False
    )

def create_test_generator():
    """Create image generator for testing."""
    return test_datagen.flow_from_directory(
        test_combined_dir,
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical',
        shuffle=False
    )

In [25]:
# Function to Extract Features from Images
def extract_features(generator, sample_count):
    """Extract features using the VGG16 model."""
    features = np.zeros(shape=(sample_count, 7, 7, 512))  
    labels = np.zeros(shape=(sample_count, generator.num_classes))
    
    i = 0
    for inputs_batch, labels_batch in generator:
        features_batch = model.predict(inputs_batch)
        features[i * generator.batch_size : (i + 1) * generator.batch_size] = features_batch
        labels[i * generator.batch_size : (i + 1) * generator.batch_size] = labels_batch
        i += 1
        if i * generator.batch_size >= sample_count:
            break
    
    return features, labels

In [26]:
# Create Generators
train_generator = create_train_generator()
validation_generator = create_validation_generator()
test_generator = create_test_generator()

# Extract Features for Training, Validation, and Testing Sets
train_features, train_labels = extract_features(train_generator, train_generator.samples)
valid_features, valid_labels = extract_features(validation_generator, validation_generator.samples)
test_features, test_labels = extract_features(test_generator, test_generator.samples)

# Reshape the features for Random Forest
train_features = np.reshape(train_features, (train_features.shape[0], 7 * 7 * 512))
valid_features = np.reshape(valid_features, (valid_features.shape[0], 7 * 7 * 512))
test_features = np.reshape(test_features, (test_features.shape[0], 7 * 7 * 512))

Found 1322 images belonging to 3 classes.
Found 60 images belonging to 3 classes.
Found 150 images belonging to 3 classes.
1/1 ━━━━━━━━━━━━━━━━━━━━ 16s 16s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 10s 10s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 10s 10s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 10s 10s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 12s 12s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 12s 12s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step
1

In [27]:
# Convert labels from categorical to numerical
train_labels_numeric = np.argmax(train_labels, axis=1)
valid_labels_numeric = np.argmax(valid_labels, axis=1)
test_labels_numeric = np.argmax(test_labels, axis=1)

# Initialize and Train the Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(train_features, train_labels_numeric)

# Save the model
joblib.dump(rf_model, 'random_forest_model.pkl')

print("Random Forest model saved as 'random_forest_model.pkl'")

Random Forest model saved as 'random_forest_model.pkl'


In [28]:
# Predict on the validation set
y_pred_val = rf_model.predict(valid_features)

# Evaluate the Model on Validation Set
print(f'Validation Accuracy: {accuracy_score(valid_labels_numeric, y_pred_val) * 100:.2f}%')
print("Validation Classification Report:\n", classification_report(valid_labels_numeric, y_pred_val))
print("Validation Confusion Matrix:\n", confusion_matrix(valid_labels_numeric, y_pred_val))

# Predict on the test set
y_pred_test = rf_model.predict(test_features)

# Evaluate the Model on Test Set
print(f'Test Accuracy: {accuracy_score(test_labels_numeric, y_pred_test) * 100:.2f}%')
print("Test Classification Report:\n", classification_report(test_labels_numeric, y_pred_test))
print("Test Confusion Matrix:\n", confusion_matrix(test_labels_numeric, y_pred_test))

Validation Accuracy: 88.33%
Validation Classification Report:
               precision    recall  f1-score   support

           0       0.79      0.95      0.86        20
           1       0.90      0.95      0.93        20
           2       1.00      0.75      0.86        20

    accuracy                           0.88        60
   macro avg       0.90      0.88      0.88        60
weighted avg       0.90      0.88      0.88        60

Validation Confusion Matrix:
 [[19  1  0]
 [ 1 19  0]
 [ 4  1 15]]
Test Accuracy: 84.00%
Test Classification Report:
               precision    recall  f1-score   support

           0       0.79      0.90      0.84        50
           1       0.91      0.84      0.88        50
           2       0.83      0.78      0.80        50

    accuracy                           0.84       150
   macro avg       0.84      0.84      0.84       150
weighted avg       0.84      0.84      0.84       150

Test Confusion Matrix:
 [[45  1  4]
 [ 4 42  4]
 [ 8  3 3

In [34]:
# Function to Predict New Input
def predict_new_image(image_path):
    """Predict the class of a new image using the Random Forest model."""
    img = load_img(image_path, target_size=(224, 224))  # Load and resize the image
    x = img_to_array(img)  # Convert image to array
    x = x.astype('float32') / 255.  # Normalize
    x = np.expand_dims(x, axis=0)  # Add batch dimension

    # Extract features using VGG16 model
    features = model.predict(x)
    features = np.reshape(features, (1, 7 * 7 * 512))  # Reshape for Random Forest input

    # Predict using Random Forest
    prediction = rf_model.predict(features)
    
    return prediction[0]  # Return the predicted class

In [35]:
# Example Usage
new_image_path = r"C:\Users\grand\Downloads\Dataset\Test\Test\Powdery\9ec7295cb1d44c2d.jpg"  # Update this with the path to the image
predicted_class = predict_new_image(new_image_path)
print(f'The predicted class for the image is: {predicted_class}')

# Optional: Mapping numerical predictions to class labels
class_labels = {0: 'Healthy', 1: 'Powdery', 2: 'Rust'}  # Replace with your actual labels
predicted_label = class_labels.get(predicted_class, 'Unknown')  # Use get to handle potential missing keys
print(f'The predicted label for the image is: {predicted_label}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 769ms/step
The predicted class for the image is: 1
The predicted label for the image is: Powdery
